Imports

In [72]:
import pandas as pd
import numpy as np

Nodes

In [73]:
class Node():
    def evaluate(self, values):
        pass
    def to_scentence(self):
        pass
    def parse(text):
        text = text.replace(' ', '')
        if text[0] == '(' and text[-1] == ')':
            text = text[1:-1]

        if '(' not in text:
            if '|' in text:
                return Disjunction(Node.parse(text[:text.index('|')]), Node.parse(text[text.index('|') + 1:]))
            elif '&' in text:
                return Conjunction(Node.parse(text[:text.index('&')]), Node.parse(text[text.index('&') + 1:]))
            elif '<->' in text:
                return Biimplication(Node.parse(text[:text.index('<->')]), Node.parse(text[text.index('<->') + 3:]))
            elif '->' in text:
                return Implication(Node.parse(text[:text.index('->')]), Node.parse(text[text.index('->') + 2:]))
            elif '!' in text:
                return Negation(Node.parse(text[1:]))
            return Variable(text, int(text[1:]))

        depth = 0
        for i in range(len(text)):
            if text[i] == '(':
                depth += 1
            elif text[i] == ')':
                depth -= 1
            elif depth == 0 and text[i] == '|':
                return Disjunction(Node.parse(text[:i]), Node.parse(text[i + 1:]))
            elif depth == 0 and text[i] == '&':
                return Conjunction(Node.parse(text[:i]), Node.parse(text[i + 1:]))
            elif depth == 0 and text[i] == '-' and text[i + 1] == '>':
                return Implication(Node.parse(text[:i]), Node.parse(text[i + 2:]))
            elif depth == 0 and text[i] == '<' and text[i + 1] == '-' and text[i + 2] == '>':
                return Biimplication(Node.parse(text[:i]), Node.parse(text[i + 3:]))
            elif depth == 0 and text[i] == '!':
                return Negation(Node.parse(text[i + 1:]))
        
        raise Exception('Something went wrong')
    
    def __str__(self):
        pass

class Variable(Node):
    def __init__(self, name, index):
        self.name = name
        self.index = index

    def evaluate(self, values):
        return values[self.index]
    
    def to_scentence(self, root = True):
        return f"{self.name}{'.' if root else ''}", 0
    
    def __str__(self):
        return self.name
    
    def __eq__(self, other):
        if isinstance(other, Variable):
            return self.name == other.name
        if isinstance(other, Negation):
            return self.name == other.expr.name
        return False
    
class Negation(Node):
    def __init__(self, expr):
        self.expr = expr

    def evaluate(self, values):
        return not self.expr.evaluate(values)
    
    def to_scentence(self, root = True):
        text, depth = self.expr.to_scentence(root = False)
        return f"!{text}{'.' if root else ''}", depth

    def __str__(self):
        return f"!{self.expr}"
    
    def __eq__(self, other):
        if isinstance(other, Negation):
            return self.expr == other.expr
        if isinstance(other, Variable):
            return self.expr.name == other.name
        return False

class Implication(Node):
    def __init__(self, left, right):
        self.left = left
        self.right = right

    def evaluate(self, values):
        return not self.left.evaluate(values) or self.right.evaluate(values)
    
    def to_scentence(self, nested = False, root = True):
        left_text, left_depth = self.left.to_scentence(nested=True, root=False) if isinstance(self.left, Implication) else self.left.to_scentence(root=False)
        right_text, right_depth = self.right.to_scentence(root=False)

        depth = max(left_depth, right_depth)

        return f"{'if ' if not nested else ''}{left_text} then{',' * depth} {right_text}{'.' if root else ''}", depth + 1

    def __str__(self):
        return f"({self.left} -> {self.right})"
    
    def __eq__(self, other):
        if isinstance(other, Implication):
            return self.left == other.left and self.right == other.right
        return False
    
class Disjunction(Node):
    def __init__(self, left, right):
        self.left = left
        self.right = right

    def evaluate(self, values):
        return self.left.evaluate(values) or self.right.evaluate(values)
    
    def to_scentence(self, root = True):
        left_text, left_depth = self.left.to_scentence(root=False)
        right_text, right_depth = self.right.to_scentence(root=False)
        depth = max(left_depth, right_depth)
        return f"{left_text} or{',' * depth} {right_text}{'.' if root else ''}", depth + 1


    def __str__(self):
        return f"({self.left} | {self.right})"
    
    def __eq__(self, other):
        if isinstance(other, Disjunction):
            return self.left == other.left and self.right == other.right
        return False
    
class Conjunction(Node):
    def __init__(self, left, right):
        self.left = left
        self.right = right

    def evaluate(self, values):
        return self.left.evaluate(values) and self.right.evaluate(values)
    
    def to_scentence(self, root = True):
        left_text, left_depth = self.left.to_scentence(root=False)
        right_text, right_depth = self.right.to_scentence(root=False)
        depth = max(left_depth, right_depth)
        return f"{left_text} and{',' * depth} {right_text}{'.' if root else ''}", depth + 1

    def __str__(self):
        return f"({self.left} & {self.right})"
    
    def __eq__(self, other):
        if isinstance(other, Conjunction):
            return self.left == other.left and self.right == other.right
        return False
    
class Biimplication(Node):
    def __init__(self, left, right):
        self.left = left
        self.right = right

    def evaluate(self, values):
        return self.left.evaluate(values) == self.right.evaluate(values)

    def to_scentence(self, root = True):
        left_text, left_depth = self.left.to_scentence(root=False)
        right_text, right_depth = self.right.to_scentence(nested = True, root=False) if isinstance(self.right, Implication) else self.right.to_scentence(root=False)
        depth = max(left_depth, right_depth)
        return f"{left_text} if and only if{',' * depth} {right_text}{'.' if root else ''}", depth + 1
    
    def __str__(self):
        return f"({self.left} <-> {self.right})"
    
    def __eq__(self, other):
        if isinstance(other, Biimplication):
            return self.left == other.left and self.right == other.right
        return False

Generator

In [74]:
def generate(depth, n):
    for i in range(n):
        yield Variable(f"p{i}", i)

    if depth == 0:
        for neg in generate(depth - 1, n):
            yield Negation(neg)
    elif depth > 0:
        for left in generate(depth - 1, n):
            for right in generate(depth - 1, n):
                if left != right:
                    yield Implication(left, right)
                    yield Disjunction(left, right)
                    yield Conjunction(left, right)
                    yield Biimplication(left, right)

print(len(list(generate(2, 4))))

150580


Find Unique Truthtable Numbers

In [75]:
def generate_number(expression, n):
    output = 0
    for i in range(2 ** n):
        values = [bool(i & (1 << j)) for j in range(n)]
        if expression.evaluate(values):
            output += 2 ** i
    return output

Generate Dataset

In [76]:
def generate_dataset(n, depth):
    data = []
    for expression in generate(depth, n):
        number = generate_number(expression, n)
        data.append([str(expression)] + [number])
    output = pd.DataFrame(data, columns=["expression", "key"])

    for key in output['key'].unique():
        output.loc[output['key'] == key, 'ratio'] = 1 / output.loc[output['key'] == key].shape[0]
    return output

In [77]:
# generate_dataset(4, 2).to_csv("dataset.csv", index=False)
test_data = pd.read_csv("dataset.csv")
test_data

FileNotFoundError: [Errno 2] No such file or directory: 'dataset.csv'

Generate premise from conclusion

In [ ]:
def generate_premise(value, indexes):
    for i in indexes:
        if i < value: continue
        for j in indexes:
            if j < value: continue
            if i != value and j != value and i > j and i & j == value:
                yield i, j, value
    

In [ ]:
value = np.random.choice(list(generate(2,4)))

print(value, value.to_scentence())
parst = Node.parse(str(value))
print(parst, parst.to_scentence())

((p3 -> !p2) & (!p1 & p3)) ('if p3 then !p2 and, !p1 and p3.', 2)
((p3 -> !p2) & (!p1 & p3)) ('if p3 then !p2 and, !p1 and p3.', 2)


In [ ]:
picked = test_data.sample(1)
lists = list(generate_premise(picked["key"].values[0], test_data["key"].unique()))
while len(lists) == 0:
    picked = test_data.sample(1)
    lists = list(generate_premise(picked["key"].values[0], test_data["key"].unique()))
a,b,c = lists[np.random.randint(0, len(lists))]
prem_0 = Node.parse(str(test_data[test_data["key"] == a]["expression"].values[0])).to_scentence()
prem_1 = Node.parse(str(test_data[test_data["key"] == b]["expression"].values[0])).to_scentence()
conc = Node.parse(str(test_data[test_data["key"] == c]["expression"].values[0])).to_scentence()

print(f'\x7b{prem_0[0]} {prem_1[0]}\x7d', f'{conc[0]}')

{if p0 or !p1 then, p3 and !p2. p1 and p2 if and only if, !p2 and !p3.} p2 if and only if, !p2 and !p3.


Tokenizer

In [ ]:
class Tokenizer():
    symbols = ['{', '}', '.', ',', '!']
    words = ['and', 'or', 'if', 'then', 'only']
    def Split(input):
        if input[0] in Tokenizer.symbols:
            return [Tokenizer.symbols.index(input[0])] + Tokenizer.Split(input[1:])
        elif input[-1] in Tokenizer.symbols:
            return Tokenizer.Split(input[:-1]) + [Tokenizer.symbols.index(input[-1])]
        elif input in Tokenizer.words:
            return [Tokenizer.words.index(input) + len(Tokenizer.symbols)]
        elif input[0] == 'p' and input[1:].isnumeric():
            return [int(input[1:]) + len(Tokenizer.words) + len(Tokenizer.symbols)]
        
        raise Exception(f'Invalid input: {input}')
    
    def Tokenize(input):
        output = []
        input_words = input.split(' ')
        for word in input_words:
            if word in Tokenizer.symbols:
                output.append(Tokenizer.symbols.index(word))
            elif word in Tokenizer.words:
                output.append(Tokenizer.words.index(word) + len(Tokenizer.symbols))
            elif word[0] == 'p' and word[1:].isnumeric():
                output.append(int(word[1:]) + len(Tokenizer.symbols) + len(Tokenizer.words))
            else:
                output += Tokenizer.Split(word)
        return output

In [ ]:
tokens = Tokenizer.Tokenize(f'\x7b{prem_0[0]} {prem_1[0]}\x7d')
print(tokens)

[0, 7, 10, 6, 4, 11, 8, 3, 13, 5, 4, 12, 2, 11, 5, 12, 7, 5, 9, 7, 3, 4, 12, 5, 4, 13, 2, 1]
